In [1]:
import requests
from getpass import getpass

# Login

In [2]:
data = {
    "email":input('email:'),
    "password":getpass('password:')
}

email:ntc61082@live.missouristate.edu
password:········


In [180]:
response = requests.post('https://bootcampspot.com/api/instructor/v1/login', json=data)

# Get Assigned Classes

In [181]:
params = {
    "Content-Type":"application/json",
    "authToken":response.json()['authenticationInfo']['authToken']
}

In [182]:
me = requests.get('https://bootcampspot.com/api/instructor/v1/me', params=params)

In [183]:
courseIds = [x['courseId'] for x in me.json()['enrollments']]
enrollmentIds = [x['id'] for x in me.json()['enrollments']]
print('courseIds',courseIds)
print('enrollmentIds',enrollmentIds)

courseIds [2192, 1513, 1084]
enrollmentIds [395037, 253348, 200149]


# Select Assigned Class
This example selects the first listed class

In [184]:
courseId = courseIds[0]
enrollmentId = enrollmentIds[0]

# Get Data

courseId is required by attenance and grades

In [185]:
json = {
    'courseId':courseId
}

In [186]:
attendance = requests.post('https://bootcampspot.com/api/instructor/v1/attendance', params=params, json=json)
grades = requests.post('https://bootcampspot.com/api/instructor/v1/grades', params=params, json=json)

enrollmentId is required by sessions and assignments

In [187]:
json = {
    'enrollmentId':enrollmentId
}

In [188]:
sessions = requests.post('https://bootcampspot.com/api/instructor/v1/sessions', params=params, json=json)
assignments = requests.post('https://bootcampspot.com/api/instructor/v1/assignments', params=params, json=json)

Example sessionId Selection

In [189]:
sessionsDict = {x['session']['name']:x['session']['id'] for x in sessions.json()['calendarSessions'] if 'Not required' not in x['session']['name']}
sessionId = sessionsDict['Dipping Into Data']

sessionId is required by sessionDetal

In [190]:
json = {
    'sessionId':sessionId
}

In [191]:
sessionDetail = requests.post('https://bootcampspot.com/api/instructor/v1/sessionDetail', params=params, json=json)

# Create Attendace CSV

In [104]:
import csv
attendance = attendance.json()

In [123]:
fieldnames = [x for x in sessionsDict]
fieldnames[0]='Name'

In [102]:
students = []
for x in attendance:
    if x['studentName'] not in students:
        students.append(x['studentName'])

In [129]:
with open('attendance.csv','w',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for student in students:
        sa = {x['sessionName']:x['present'] or x['remote'] for x in attendance if x['studentName'] == student and 'Not required' not in x['sessionName'] and x['sessionName'] in fieldnames}
        sa['Name'] = student
        writer.writerow(sa)

# Create Grades CSV

In [192]:
import csv
grades = grades.json()
assignments = assignments.json()

In [168]:
fieldnames = ['Name']
fieldnames.extend([x['title'] for x in assignments['calendarAssignments'] if 'Milestone' not in x['title']])

In [171]:
students = []
for x in grades:
    if x['studentName'] not in students:
        students.append(x['studentName'])

In [179]:
with open('grades.csv','w',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for student in students:
        sg = {x['assignmentTitle']:x['grade'] for x in grades if x['studentName'] == student and 'Milestone' not in x['assignmentTitle']}
        sg['Name'] = student
        writer.writerow(sg)